In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

In [ ]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_Fashion/")

In [ ]:
lr = 0.0002
batch_size = 32
epochs = 100000

image_dimension = 784
neural_network_dimension = 128
latent_variable_dimension = 10
z_noise_dim = 10

def xavier(in_shape):
  val = tf.random_normal(shape = in_shape, stddev = 1./tf.sqrt(in_shape[0]/2.))
  return val

In [ ]:
Disc_W = {"disc_H" : tf.Variable(xavier_init([latent_variable_dimension, neural_network_dimension])),
          "disc_final" : tf.Variable(xavier_init([neural_network_dimension, 1]))
          }

Disc_ Bias = {"disc_H" : tf.Variable(xavier_init([neural_network_dimension])),
          "disc_final" : tf.Variable(xavier_init([1]))
          }
         

In [ ]:
AE_weight = {"E_weight_encoder" : tf.Variable(xavier_init([image_dimension, neural_network_dimension])),
             "E_weight_hidden" : tf.Variable(xavier_init([neural_network_dimension, latent_variable_dimension])),


             "D_weight_hidden" : tf.Variable(xavier_init([latent_variable_dimension, neural_network_dimension])),
             "D_weight_decoder" : tf.Variable(xavier_init([neural_network_dimension, image_dimension]))
          }

AE_bias = {"E_bias_encoder" : tf.Variable(xavier_init([neural_network_dimension])),
           "E_bias_hidden" : tf.Variable(xavier_init([latent_variable_dimension])),

           "D_bias_hidden" : tf.Variable(xavier_init([neural_network_dimension])),
           "D_bias_decoder" : tf.Variable(xavier_init([image_dimension]))
          }          

In [ ]:
#placeholders for the inputs

Z_input = tf.placeholder(tf.float32, shape = [None, z_noise_dim], name = "input_noise")
X_input = tf.placeholder(tf.float32, shape = [None, image_dimension], name = "real_input")

def Encoder(x):
  hidden_layer = tf.nn.relu(tf.add(tf.matmul(x, AE_weight['E_weight_encoder']), AE_bias['E_bias_encoder']))
  encoder_output = tf.add(tf.matmul(hidden_layer, AE_weight['E_weight_hidden']), AE_bias['E_bias_hidden'])
  return encoder_output


def Decoder(x):
  hidden_layer = tf.nn.relu(tf.add(tf.matmul(x, AE_weight['D_weight_hidden']), AE_bias['D_bias_hidden']))
  decoder_output = tf.add(tf.matmul(hidden_layer, AE_weight['D_weight_decoder']), AE_bias['D_bias_decoder'])
  prob = tf.nn.sigmoid(decoder_output)
  return prob, decoder_output


def Discriminator(x):
  hidden_layer = tf.nn.relu(tf.add(tf.matmul(x, Disc_W['disc_H']), Disc_Bias['disc_H']))
  final_output = tf.add(tf.matmul(hidden_layer, Disc_W['disc_final']), Disc_Bias['disc_final'])
  disc_op = tf.nn.sigmoid(final_output)
  return disc_op

In [ ]:
#build the graph

latent_output = Encoder(X_input)
_, final_output = Decoder(latent_output)

real_output_Disc = Discriminator(Z_input) #prior gaussian, bernoulli or uniform distribution
fake_output_Disc = Discriminator(latent_output)

In [ ]:
AE_Loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits = final_output, labels = X_input))

Discriminator_Loss = -tf.reduce_mean(tf.log(real_output_Disc) + tf.log(1.0 - fake_output_Disc))
Generator_Loss = -tf.reduce_mean(tf.log(fake_output_Disc))

#tensorboard summary
tf.summary.scalar("AE_loss", AE_Loss)
tf.summary.scalar("Disc_Total_loss", Discriminator_Loss)
tf.summary.scalar("Gen_loss", Generator_Loss)
tf.summary.histogram("Gen_loss", latent_output)

summary1 = tf.summary.merge_all()

In [ ]:
Encoder_var = [AE_weight['E_weight_encoder'], AE_bias['E_bias_encoder'], AE_weight['E_weight_hidden'], AE_bias['E_bias_hidden']]
Decoder_var = [AE_weight['D_weight_hidden'], AE_bias['D_bias_hidden'], AE_weight['D_weight_decoder'],  AE_bias['D_bias_decoder']]
Discriminator_var = [Disc_W['disc_H'], Disc_Bias['disc_H'], Disc_W['disc_final'], Disc_Bias['disc_final']]


In [ ]:
AutoEncoder_optimize = tf.train.AdamOptimizer(learning_rate = lr).minimize(AE_loss, var_list = Encoder_var + Decoder_var)
Discriminator_optimize = tf.train.AdamOptimizer(learning_rate = lr).minimize(Discriminator_loss, var_list = Discriminator_var)
Generator_optimize = tf.train.AdamOptimizer(learning_rate = lr).minimize(Generator_loss, var_list = Encoder_var)

In [1]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)
writer = tf.summary.FileWriter("./log", sess.graph)

for i in range(epochs):
  X_batch, _ = mnist.train.next_batch(batch_size)
  Z_noise = np.random.normal(0, 1, [batch_size, z_noise_dim])

  _, AE_loss_epoch = sess.run([AutoEncoder_optimize, AE_Loss], feed_dict = {X_input: X_batch})
  _, Disc_loss_epoch = sess.run([Discriminator_optimize, Discriminator_Loss], feed_dict = {X_input: X_batch, Z_input : Z_noise})
  _, Gen_loss_epoch = sess.run([Generator_optimize, Generator_Loss], feed_dict = {X_input: X_batch})

  writer.add_summary(summary.epoch)

if i%2000 == 0:
    print('Steps: {0}, AE Loss: {1}, Generator Loss: {2}, Discriminator Loss: {3}'.format(i, AE_loss_epoch, Gen_loss_epoch, Disc_loss_epoch))

In [ ]:
#Visualising the output obtained ----

test_output , _ = Decoder(Z_input)
n = 6
canvas = np.empty((28*n, 28*n))

for i in range(n):
  Z_noise = np.random.normal(0, 1, [batch_size, z_noise_dim])
    #generated_latent_layer = np.random.normal(0, 1, size = [batch_size, latent_variable_dimension])
  generated_image  = sess.run(test_output, feed_dict = {Z_input : Z_noise})
  for j in range(n):
    canvas[i * 28: (i+1) * 28, j * 28: (j+1) * 28] = generated_image[j].reshape([28, 28])


plt.figure(figsize = (n, n))
plt.imshow(canvas, origin = "upper", cmap = 'gray')
plt.show()

In [ ]:
#in the log files go and check the tensorboard logs and open the needed link in browser
# u can view each of the losses and also histogram of the approximated latent space, here a normal dist